In [244]:
import pandas as pd
import numpy as np
import re, math
from string import punctuation

In [245]:
df = pd.read_excel("./data/Eni_Shell_data.xlsx")

In [246]:
df.shape

(1586, 46)

In [247]:
df.columns

Index(['Oil Spill ID', 'Company', 'JIV Number', 'Date Reported (Shell, Eni)',
       'Year', 'JIV Date (Shell)',
       'Facility equipment/ Incident site (Shell, Eni)',
       'Area/ Terrain (Shell, Eni)', 'Cause (Shell, Eni)',
       'Volume (Shell, Eni) barrels\n', 'Clean-up Status (Shell)',
       'Comments (Shell)', 'LGA (Eni)', 'JIV Asset Id', 'Included', 'JIV Url',
       'JIV URL hyperlinked', 'JIV Cause Verified', 'Date Incident',
       'Date start investigation', 'Description of Leak Point',
       'Incident caused by', 'Incident caused by (Clean)',
       'Comments cause (amnesty classification)',
       'Cause (amnesty classification)', 'JIV Location Verified',
       'Location.Type', 'Location Unit', 'Lat/Northing', 'Long/Easting',
       'Tranformation notes', 'Latitude (normalised)',
       'Longitude (normalised)', 'Area', 'AreaUnit', 'JIV Comment Type',
       'JIV Comment', 'photo_lookup_id', 'Photo Asset Id', 'Included.1',
       'Photo Url', 'Photo url hyperlinked'

### I. Nornalize column names

In [248]:
column_names = [
    "oil_spill_id",
    "company", 
    "jiv_number",
    "date_reported",
    "year",
    "date_jiv_shell",
    "facility_equipment",
    "terrain",
    "cause",
    "barrels",
    "cleanup_status_text",
    "comments_shell",
    "lga_eni",
    "jiv_asset_id",
    "in_decoders",
    "jiv_url",
    "jiv_url_hyperlinked",
    "cause_jiv_verified",
    "date_incident",
    "date_investigation_start",
    "leak_point_text",
    "cause_incident_caused_by_dirty",
    "cause_incident_caused_by",
    "cause_amnesty_comment",
    "cause_amnesty",
    "location_jiv_verified",
    "location_type",
    "location_unit",
    "lat_northing",
    "long_eastling",
    "location_transformation_notes",
    "lat",
    "long",
    "area_decoders",
    "area_unit",
    "comment_type_jiv",
    "comment_jiv",
    "photo_lookup_id",
    "photo_asset_id",
    "in_decoders2",
    "photo_url",
    "photo_url_hyperlinked",
    "damage_photo",
    "damage_photo_followup",
    "comment_jiv_duplicate",
    "comment_jiv_text"
]

In [249]:
pd.DataFrame({"Original Column Names": df.columns, "Alias": column_names}).to_csv("columns.csv")

In [250]:
df.columns.shape, len(column_names)

((46,), 46)

In [251]:
df.columns = column_names

### II. Explore data

#### 1. Do "Included" and "Included.1" hold the same data?

In [252]:
df[["in_decoders", "in_decoders2"]].head()

,in_decoders,in_decoders2
0,Y,Y
1,Y,Y
2,N,Y
3,N,Y
4,Y,Y


In [253]:
df.in_decoders2.value_counts()

Y                                       1558
N (asset error on shell/eni wensite)      13
Missing photo                              8
N                                          5
N, photo missing                           2
Name: in_decoders2, dtype: int64

#### 2. explore Facility Equiptment

- '' inches
- pipeline name
- "at" location

In [254]:
df.facility_equipment.head(5)

0                            24'' Ogoda/Brass Pipeline
1    12'' Imo River 1 and 2 - Ogale Pipeline at komkom
2              Adibawa Well 8 S/L Wellhead at Edagberi
3           24'' Bomu - Bonny Pipeline at Okolo Launch
4       10'' Diebu Creek-Nun River Pipeline at Oporoma
Name: facility_equipment, dtype: object

In [255]:
# regexs
inch_single_quote_regex = re.compile(r"(\d+)''")
inch_double_quote_regex = re.compile(r'(\d+)"')
location_regex = re.compile(r"at\s(.*)")

inches = [np.nan] * df.shape[0]
facility_type_name = [np.nan] * df.shape[0]
facility_location = [np.nan] * df.shape[0]

no_inch_cnt = 0
missing_loc_count = 0
for i in range(df.shape[0]):
    
    facility_info = df.facility_equipment.iloc[i].lower()
    # a. exctract inches
    try:
        inches[i] = int(re.search(inch_single_quote_regex, facility_info).group(1))
    except:
        try:
            inches[i] = int(re.search(inch_double_quote_regex, facility_info).group(1))
        except:
            no_inch_cnt += 1
            
    # b. extract facility type
    type_found = False
    otherline_seen = "line" in facility_info
    flowlines = set(["flowline", "fl"])
    flowline_seen = flowlines.intersection(set(facility_info.split(" "))) or "flow line" in facility_info
    pipeline_seen = "pipeline" in facility_info
    well_seen = "well" in facility_info
    wellhead_seen = "wellhead" in facility_info or "well head" in facility_info
    manifold_seen = "manifold" in facility_info
    trunklines = set(["trunkline", "tl"])
    trunkline_seen = trunklines.intersection(set(facility_info.split(" "))) or "trunk line" in facility_info
    deliverylines = set(["deliveryline", "dl"])
    deliveryline_seen = deliverylines.intersection(set(facility_info.split(" "))) or "delivery line" in facility_info
    bulklines = set(["bulkline", "bl"])
    bulkline_seen = bulklines.intersection(set(facility_info.split(" "))) or "bulk line" in facility_info
    flowstation_seen = "flowstation" in facility_info or "flow station" in facility_info
 
    if otherline_seen:
        facility_type_name[i] = "other line"
        type_found = True
    
    if pipeline_seen and not well_seen:
        facility_type_name[i] = "pipeline"
        type_found = True
       
    if flowline_seen and "well" in facility_info:
        facility_type_name[i] = "flowline, well"
        type_found = True
        
    if flowline_seen and not well_seen:
        facility_type_name[i] = "flowline"
        type_found = True
        
    if well_seen and not wellhead_seen and not flowline_seen and not pipeline_seen:
        facility_type_name[i] = "well"
        type_found = True
        
    if wellhead_seen:
        facility_type_name[i] = "wellhead"
        type_found = True
        
    if manifold_seen:
        facility_type_name[i] = "manifold"
        type_found = True

    if trunkline_seen:
        facility_type_name[i] = "trunkline"
        type_found = True

    if deliveryline_seen:
        facility_type_name[i] = "deliveryline"
        type_found = True
        
    if bulkline_seen:
        facility_type_name[i] = "bulkline"
        type_found = True
    
    if flowstation_seen:
        facility_type_name[i] = "flowstation"
        type_found = True
    
    if not type_found:
        facility_type_name[i] = "other"
        
    # c. extract location
    try:
        facility_location[i] = str(re.search(location_regex, facility_info).group(1)).strip(punctuation)
    except:
        missing_loc_count +=1

df["inches"] = pd.Series(inches)
df["facility_type"] = pd.Series(facility_type_name)
df["facility_location"] = pd.Series(facility_location)

In [256]:
df.facility_type.value_counts()

pipeline          933
flowline, well    179
flowline          149
wellhead           64
well               52
other              46
trunkline          45
deliveryline       45
manifold           31
flowstation        21
other line         14
bulkline            7
Name: facility_type, dtype: int64

In [257]:
df.facility_location.isnull().sum()

763

In [258]:
df[["facility_location", "lga_eni"]].head()

,facility_location,lga_eni
0,NaN,Abua/Odual
1,komkom,NaN
2,edagberi,NaN
3,okolo launch,NaN
4,oporoma,NaN


#### * check if when Shell location is given, Eni is missing and vice versa

In [259]:
# shell location -> facility_location
# eni location -> lga_eni
shell_loc = [0] * df.shape[0]
eni_loc = [0] * df.shape[0]

for i in range(df.shape[0]):
    shell_cur_loc = df.facility_location.iloc[i]
    eni_curr_loc = df.lga_eni.iloc[i]
    if type(shell_cur_loc) != float:
        shell_loc[i] = 1
    if type(eni_curr_loc) != float:
        eni_loc[i] = 1
        
shell_loc_series = pd.Series(shell_loc)
eni_loc_series = pd.Series(eni_loc)
loc_series = shell_loc_series.add(eni_loc_series)
loc_series.value_counts()

1    1520
0      61
2       5
dtype: int64

In [260]:
df["location_joint"] = pd.Series(loc_series)

In [261]:
df[["oil_spill_id", "company","facility_location", "lga_eni"]][df.location_joint == 2]

,oil_spill_id,company,facility_location,lga_eni
752,385,NAOC,the mini manifold,Southern Ijaw
1156,771,NAOC,olugbobiri,Southern Ijaw
1363,703,NAOC,obama 3 deep slot,Southern Ijaw
1372,1494,NAOC,oshie flow station,Oshie
1373,1202,NAOC,obiafu 14 well location,Ogba/Ndoni/Egbema


In [262]:
df[["oil_spill_id", "company","facility_location", "lga_eni"]][df.location_joint == 1].head()

,oil_spill_id,company,facility_location,lga_eni
0,1272,NAOC,NaN,Abua/Odual
1,221,SPDC,komkom,NaN
2,640,SPDC,edagberi,NaN
3,1109,SPDC,okolo launch,NaN
4,486,SPDC,oporoma,NaN


#### * clean the facility_locations for ENI

In [263]:
df.facility_location[df.location_joint == 2] = np.nan

/home/katia/.virtualenvs/securaxis/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [264]:
df[["oil_spill_id", "company","facility_location", "lga_eni"]][df.location_joint == 2]

,oil_spill_id,company,facility_location,lga_eni
752,385,NAOC,NaN,Southern Ijaw
1156,771,NAOC,NaN,Southern Ijaw
1363,703,NAOC,NaN,Southern Ijaw
1372,1494,NAOC,NaN,Oshie
1373,1202,NAOC,NaN,Ogba/Ndoni/Egbema


In [265]:
del df["location_joint"]

#### * aggregate Shell and Eni locations in the same column

#### * extract the pipeline name

In [281]:
# handles from and to loc in cases such as: "nun-river kolo creek" or "biedu - nun-river"
def adjust_splits(splitted_list):
    from_loc = ""
    to_loc = ""
    if splitted_list[1] == "river" or splitted_list[1] == "creek":
        from_loc = splitted_list[0] + " " + splitted_list[1]
        to_loc = " ".join(splitted_list[2:])
    else:
        from_loc = splitted_list[0]
        to_loc = " ".join(splitted_list[1:])
    return from_loc, to_loc

In [284]:
df["facility_equipment_lower"] = df.facility_equipment.str.lower()
facility_names = [np.nan] * df.shape[0]
facility_start = [np.nan] * df.shape[0]
facility_end = [np.nan] * df.shape[0]

missing_fac_name_cnt = 0
for i in range(df.shape[0]):
    facility = df.facility_equipment_lower.iloc[i]
    current_facility_type = df.facility_type.iloc[i]
    if current_facility_type != "other":
        current_inches = df.inches.iloc[i]
        curr_type__abbr = current_facility_type[:5]
        # find the start index of the substring
        if math.isnan(current_inches):
            # start extracting from index 0
            start_index = 0
        else:
            # start extracting from the index of the word after the first space
            start_index = facility.find(" ") + 1
        # find the end index of the substring
        end_index = facility.find(curr_type__abbr)
        if end_index != -1:
            current_facility_name = facility[start_index:end_index-1]
            current_facility_name = current_facility_name.replace("  ", " ") # replace double space with single
            facility_names[i] = current_facility_name.strip()
            
            if current_facility_type not in ["well", "wellhead", "flowline", "flowline, well"]:

                dash_split = current_facility_name.split("-")
                to_split = current_facility_name.split(" to ")
                baskslash_split = current_facility_name.split("/")

                from_loc = ""
                to_loc = ""
                if len(dash_split) > 1:
    #                 if current_facility_name == "nun-river - kolo creek":
    #                     print("yes", dash_split, list(map(str.strip, dash_split)))
                    from_loc, to_loc = adjust_splits(list(map(str.strip, dash_split)))

                if len(baskslash_split) > 1:
                    from_loc, to_loc = adjust_splits(list(map(str.strip, baskslash_split)))

                if len(to_split) > 1:
                    from_loc, to_loc = adjust_splits(list(map(str.strip, to_split)))


                facility_start[i] = from_loc
                facility_end[i] = to_loc
            
        else:
            missing_fac_name_cnt += 1
            
print("There are", missing_fac_name_cnt, "missing facility names.")
df["facility_name"] = pd.Series(facility_names).values
df["facility_start"] = pd.Series(facility_start).values
df["facility_end"] = pd.Series(facility_end).values

There are 61 missing facility names.


In [285]:
del df["facility_equipment_lower"]
df[["facility_equipment", "inches", "facility_name", "facility_start", "facility_end", "facility_type", "facility_location"]].to_csv("./data/EniShell_transformed.csv")